# Global Slum Detector — Batch Generation (15–20 samples)
Run this in Google Colab to generate overlays for a subset of images.

In [ ]:
#@title Setup (clone + install)
!git clone https://github.com/Akila-Wasalathilaka/Slum-detection-model-using-UNET.git -q
%cd Slum-detection-model-using-UNET
!pip install -q -r requirements.txt

In [ ]:
#@title Configure paths
from pathlib import Path
import random, os, json, cv2, numpy as np
from google.colab import files

# Upload your checkpoint if not in repo
CHECKPOINT = 'best_global_model.pth'  #@param {type: 'string'}
IMAGES_DIR = 'data/test/images'       #@param {type: 'string'}
OUTPUT_DIR = 'colab_outputs'          #@param {type: 'string'}
N_SAMPLES = 20                         #@param {type: 'number'}

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
image_paths = sorted([str(p) for p in Path(IMAGES_DIR).glob('*.tif')] + [str(p) for p in Path(IMAGES_DIR).glob('*.png')] + [str(p) for p in Path(IMAGES_DIR).glob('*.jpg')])
assert len(image_paths) > 0, 'No images found in data/test/images. Upload or mount your data.'

subset = random.sample(image_paths, min(N_SAMPLES, len(image_paths)))
len(subset)

In [ ]:
#@title Batch inference and overlay generation
import cv2, numpy as np
from global_slum_detector import GlobalSlumDetector

detector = GlobalSlumDetector(CHECKPOINT)
results = []
for i, img_path in enumerate(subset, 1):
    out = detector.predict_global(img_path, use_tta=True, use_tent=True, adaptive_threshold=True)
    base = Path(img_path).stem
    overlay_path = str(Path(OUTPUT_DIR)/f'{base}_overlay.jpg')
    prob_path = str(Path(OUTPUT_DIR)/f'{base}_prob.jpg')
    cv2.imwrite(overlay_path, cv2.cvtColor(out['overlay'], cv2.COLOR_RGB2BGR))
    cv2.imwrite(prob_path, (out['probability']*255).astype(np.uint8))
    results.append({"image": img_path, "overlay": overlay_path, "prob": prob_path, "threshold": float(out['threshold']), "mean_conf": float(out['confidence'].mean())})
    print(f'[{i}/{len(subset)}] Saved: {overlay_path}')

with open(Path(OUTPUT_DIR)/'batch_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print('Done. Results saved to', OUTPUT_DIR)

In [ ]:
#@title Download results as zip
import shutil
shutil.make_archive('overlays', 'zip', OUTPUT_DIR)
files.download('overlays.zip')